In [ ]:
import requests
import json
import time
import pandas as pd
import google.generativeai as genai


def run_gemini(model_name, prompt):
    print("O run_gemini está sendo usado!")
    print(f"O prompt que está sendo usado é: {prompt}!")
    generation_config = {
    "candidate_count": 1,
    }
    safety_settings = {
    "HARASSMENT": "BLOCK_NONE",
    "HATE": "BLOCK_NONE",
    "SEXUAL": "BLOCK_NONE",
    "DANGEROUS": "BLOCK_NONE",
    }
    testeFinal = tokenAPI1
    teste = tokenAPI2
    mae = tokenAPI3
    eu = tokenAPI4
    pai1 = tokenAPI5
    pai2 = tokenAPI6
    jogos = tokenAPI7
    listaAPIs = [testeFinal, teste, mae, eu, pai1, pai2, jogos]
    nomesAPIs = ["testeFinal", "teste", "mae", "eu", "pai1", "pai2", "jogos"]
    index = 0
    nomeModelo = model_name.replace("google/", "models/")
    while len(listaAPIs) >= 1:
        GOOGLE_API_KEY = listaAPIs[index]
        nomeAPI = nomesAPIs[index]
        print(f"A API é: {nomeAPI}")
        print()
        print()
        genai.configure(api_key=GOOGLE_API_KEY)
        model = genai.GenerativeModel(model_name=nomeModelo, generation_config=generation_config, safety_settings=safety_settings)
        try:
            resposta_completa = model.generate_content(prompt)
            resposta = resposta_completa.text
        except Exception as e:
            print("Erro inesperado")
            listaAPIs.pop(0)
            nomesAPIs.pop(0)
            # time.sleep(20)
            continue
        break

    if len(listaAPIs) == 0:
        print("Nenhuma API restante")
        return False

    l = list()
    l.append(dict())
    l[0]['generated_text'] = resposta

    time.sleep(30)
    return l

In [ ]:
# https://www.youtube.com/watch?v=jsCUDeg_Op4&t=185s

!pip install -q -U transformers accelerate bitsandbytes

In [ ]:
from huggingface_hub import login
login(token=tokenHuggingFace)
print('Deu certo')

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM, pipeline

import shutil
import torch
import gc
import time
import transformers
from transformers import BitsAndBytesConfig

from accelerate import dispatch_model

def load_model(model_name, quant):
    # Enable quantization using bitsandbytes (e.g., 4-bit)
    if quant:
      quantization_config = BitsAndBytesConfig(
          load_in_4bit=True,  # Enable 4-bit quantization
          bnb_4bit_compute_dtype=torch.float16,  # Use float16 for computations
          bnb_4bit_use_double_quant=True,  # Use double quantization for memory efficiency
          bnb_4bit_quant_type="nf4"  # Use normalized float4 for better accuracy
      )
    else:
      quantization_config = BitsAndBytesConfig(
          load_in_4bit=False,  # Enable 4-bit quantization
          bnb_4bit_compute_dtype=torch.float32,  # Use float16 for computations
          bnb_4bit_use_double_quant=False,  # Use double quantization for memory efficiency
          bnb_4bit_quant_type="nf4"  # Use normalized float4 for better accuracy
      )

    start_time = time.time()

    # Load tokenizer and quantized model
    if model_name == "google/flan-t5-small":
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSeq2SeqLM.from_pretrained(
            model_name,
            quantization_config=quantization_config,
            device_map="auto",  # Automatically map the model to available devices
        )
    else:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=quantization_config,
            device_map="cuda",  # Automatically map the model to available devices
        )

    # Create the pipeline
    pipe = pipeline(
       "text2text-generation" if model == "google/flan-t5-small" else "text-generation",
        model=model,
        tokenizer=tokenizer,
    )

    end_time = time.time()
    elapsed_time = end_time - start_time

    print('MODEL LOADED!', elapsed_time, "secs")
    return pipe, model

def get_gpu_memory():
    """Retorna o uso da memória da GPU."""
    return torch.cuda.memory_allocated() / (1024 ** 2)  # Em MB

def get_disk_usage():
    """Retorna o uso do espaço em disco do cache do Hugging Face."""
    cache_path = "/root/.cache/huggingface"
    if os.path.exists(cache_path):
        return shutil.disk_usage(cache_path).used / (1024 ** 2)  # Em MB
    return 0

def unload_model(model, pipe):
    """Libera memória da GPU e do disco, e exibe os resultados."""

    print("\n📌 **Antes da liberação:**")
    gpu_before = get_gpu_memory()
    disk_before = get_disk_usage()
    print(f"🔹 Memória da GPU usada: {gpu_before:.2f} MB")
    print(f"🔹 Espaço usado no cache do Hugging Face: {disk_before:.2f} MB")

    # 🔽 Libera a GPU
    if model is not None:
        model.to("cpu")  # Move para CPU
        del model  # Remove referência ao modelo

    if pipe is not None:
        del pipe  # Remove referência ao pipeline

    gc.collect()  # Força a coleta de lixo
    torch.cuda.empty_cache()  # Limpa a VRAM

    print("\n✅ **Após limpar a GPU:**")
    gpu_after = get_gpu_memory()
    print(f"🔹 Memória da GPU usada: {gpu_after:.2f} MB (Deveria diminuir!)")

    # 🔽 Remove cache do Hugging Face
    cache_path = "/root/.cache/huggingface"
    if os.path.exists(cache_path):
        shutil.rmtree(cache_path)

    print("\n✅ **Após remover o cache do Hugging Face:**")
    disk_after = get_disk_usage()
    print(f"🔹 Espaço usado no cache do Hugging Face: {disk_after:.2f} MB (Deveria ser 0!)")

In [ ]:
def small_test(pipe):
    # Example usage
    prompt = "What is 2 + 2?"
    result = pipe(prompt, max_length=200, num_return_sequences=1)
    print(result)


In [ ]:
def build_prompt(question, dica):
    prompt = ""
    answer_separator = "."
    prompt = "Responda à seguinte questão de múltipla escolha no formato 'Alternativa: LETRA, justificativa'. Com LETRA devendo ser substituída pela letra da alternativa que você acha correta ou a mais correta dentre as alternativas da questão. Caso não considere nenhuma certa, substitua LETRA por X e justificativa devendo ser substituída pelo raciocínio que te levou a escolher a LETRA.\n"
    if dica:
        prompt += f'''Utilize exclusivamente o ferramental juridico fornecido para respondê-la. Não útilize outras leis além das fornecidas.\n
                     -Ferramental Jurídico:
                     {question[legalTools]}\n'''
    prompt += "Segue a questão:\n"
    prompt += "Questão: \n" + question["question"] + "\n"
    return prompt
    
    print('Prompt built.')


In [ ]:
import re

def map_multiple_choice(text):
    """
    Parses a text to map multiple-choice letters to their answers and finds the letter inside the boxed notation.

    Args:
        text (str): Input text containing multiple-choice options and a boxed answer.

    Returns:
        dict: A dictionary mapping letters to answers.
        str: The letter corresponding to the boxed answer.
    """
    # Regular expression to extract options like "A. 1", "B. 3", etc.
    options = re.findall(r"([A-E])\.\s*([\d\.]+)", text)
    
    # Create a mapping of letter -> answer
    # letter_to_answer = {letter: float(answer) for letter, answer in options}
    letter_to_answer = {}
    for letter, answer in options:
        try:
            letter_to_answer[letter] = float(answer)
        except Exception as e:
            print("Não foi possível fazer o mapeamento!!")
            return "X"

    # Extract the boxed answer using \boxed{}
    boxed_match = re.search(r"\\boxed\{([\d\.]+)\}", text)
    if boxed_match:
        boxed_answer = float(boxed_match.group(1))
    else:
        return 'X'
        
    # Find the letter corresponding to the boxed answer
    boxed_letter = None
    for letter, answer in letter_to_answer.items():
        if answer == boxed_answer:
            boxed_letter = letter
            break
    if boxed_letter is None:
        return 'X'
    return boxed_letter

def parse_answer(full_answer):
    listaPadroes = ""
    padroesPortugues = [
        r"Alternativa:\s+([a-dA-D])",
        r"\*?\*?Resposta\*?\*?:\*?\*?\s*\*?\*?([a-d])\*?\*?",
        r"\*?Resposta:\*?\s*\*?([a-d])\*?",
        r"\*?resposta\s*\*?é\s*\*?:\*?\s*\*?([a-d])\*?",
        r"\*?a\s*\*?opção\s*\*?correta\s*\*?é\s*\*?:\*?\s*\*?([a-d])\*?",
        r"\\boxed\{(?:\\text\{(\*?[a-d]\*?)\}|(\*?[a-d]\*?)[^a-d]*)\}",
        r"resposta é\s*\*?([a-d])\*?",
        r"a opção correta é a\s*\*?([a-d])\*?",
        r"a opção correta é\s*\*?([a-d])\*?",
        r"\\boxed\{([a-d])\}",
        r"resposta\s*:\s*\[([a-d])\]",
        r"opção correta\s+é\s+(?:a\s+)?(?:opção\s+)?([a-d])",
        r"resposta correta\s+é\s+(?:a\s+)?([a-d])",
    ]
    padroesIngles = [
        r"Alternativa:\s+([a-dA-D])",
        r"\*?\*?Answer\*?\*?:\*?\*?\s*\*?\*?([a-d])\*?\*?",
        r"\*?Answer:\*?\s*\*?([a-d])\*?",
        r"\*?answer\s*\*?is\s*\*?:\*?\s*\*?([a-d])\*?",
        r"\*?the\s*\*?correct\s*\*?option\s*\*?is\s*\*?:\*?\s*\*?([a-d])\*?",
        r"\\boxed\{(?:\\text\{(\*?[a-d]\*?)\}|(\*?[a-d]\*?)[^a-d]*)\}",
        r"answer is\s*\*?([a-d])\*?",
        r"the correct option is\s*\*?([a-d])\*?",
        r"\\boxed\{([a-d])\}",
        r"answer\s*=\s*([a-d])",
        r"(?:Therefore, the correct answer is\s*\**([a-d])\.*\**)",
        r"(?:Therefore, the correct answer is:\s*\n\**([a-d])\.*\**)",
    ]
    listaPadroes = [padroesPortugues, padroesIngles]
    for tipoPadrao in listaPadroes:
        for padrao in tipoPadrao:
            match = re.search(padrao, full_answer, re.IGNORECASE)
            if match:
                return match.group(1)  # Retorna a primeira correspondência encontrada
    
    return map_multiple_choice(full_answer)  # Retorna None se nenhum padrão for encontrado

In [ ]:
import os
import zipfile

# Define the directory and the archive name
directory = '/kaggle/working'

def zip_answers():
    output_zip = f'answers.zip'
    # Create the zip archive
    with zipfile.ZipFile(output_zip, 'w') as zipf:
        for root, _, files in os.walk(directory):
            num_files = len(files)
            for file in files:
                if file.startswith('answers_'):  # Filter files starting with "answers_"
                    file_path = os.path.join(root, file)
                    # Add the file to the zip archive, maintaining its directory structure
                    arcname = os.path.relpath(file_path, directory)
                    zipf.write(file_path, arcname)

        print(f"Zip archive {output_zip} created successfully!")
        return output_zip


    from IPython.display import FileLink
    return FileLink(output_zip)

zip_answers()


In [ ]:
from fractions import Fraction

def is_equal(n1, n2):
    # Convert n1 and n2 to Fraction if they are strings representing rational numbers
    if n1 == n2:
        return True
    else:
        return False


In [ ]:
def extract_last_digit(text):
    match = re.search(r'-(\d+)$', text)
    if match:
        return match.group(1)
    return None

print('Done.')

In [ ]:
import time

# https://huggingface.co/docs/transformers/v4.27.1/pipeline_tutorial#using-pipelines-on-a-dataset
def prompts(questions, dica):
    print("O prompts está sendo usado!")
    for question in questions:
       prompt = build_prompt(question, dica)
       yield prompt

def call_gemini(questions, model_name, dica):
    print("O call_gemini está sendo usado!")
    for question in questions:
       prompt = build_prompt(question, dica)
       yield run_gemini(model_name, prompt)

def do_exam(questions_df, model_name, quant, pipe, temperature, prova, dica):
    tipo_arquivo = ""
    if dica:
        tipo_arquivo = "COM DICAS"
    else:
        tipo_arquivo = "SEM DICAS"
    total_time = 0
    questions = questions_df.to_dict(orient='records')
    print(f"A prova é: {prova}")

    question_numbers = list()
    question_prompts = list()
    question_bodies = list()

    question_labels = list()
    question_math_levels = list()
    question_interpretation_levels = list()
    question_types = list()
    question_groups = list()

    languages = list()
    
    llm_full_answers = list()
    token_size_questions = list()
    token_size_responses = list()

    correct_answers = list()
    is_correct_list = list()
    times = list()
    llm_letters = list()

    prompt_types = list()

    temperatures = list()

    question_seq = -1
    start_time = time.time()

    if model_name == 'google/gemini-1.5-flash' or  \
            model_name == 'google/gemini-1.5-flash-8b' or \
            model_name == "google/gemini-2.0-flash-exp" or \
            model_name == "google/gemini-1.5-pro" or \
            model_name == "google/gemini-1.0-pro":
        l = call_gemini(questions, model_name, dica)
    else:
        l = pipe(prompts(questions, dica), 
                 truncation=True, max_length = 1000, 
                 num_return_sequences=1)

    last_question_s = 0
    continuar = True
    nome_arq = ''
    for output in l:

       if output is False:  # Se run_gemini retornar False, salvar os dados e sair
            print("Erro detectado! Salvando as respostas já obtidas...")
            if dica:
                df = pd.DataFrame({
                    'question_number': questions_df['Número'][:(question_seq + 1)],  
                    'question_body': questions_df['question'][:(question_seq + 1)],
                    'full_model_name': model_name,
                    'llm_chosen_answer': llm_letters,
                    'correct_answer': correct_answers,
                    'is_correct': is_correct_list,
                    'legalTools':questions_df["legalTools"][:(question_seq + 1)],
                    'Anulada?': questions_df['Anulada?'][:(question_seq + 1)],
                    'question_prompt': question_prompts,
                    'response': llm_full_answers,
                    'token_size_question': token_size_questions,
                    'token_size_response': token_size_responses,
                    'elapsed_time_sec': times
                })
            model_name = model_name.replace("/", ".")
            nome_arq = prova + f" {tipo_arquivo} - Modelo: " + model_name 
            df.to_csv(f'{nome_arq}_atéQuestão{question_seq}.csv')
            continuar = False
            return total_time, is_correct_list.count(True), continuar        
       question_seq += 1
       question = questions[question_seq]

       prompt = build_prompt(question, dica)
       full_answer = output[0]['generated_text']

       print("\n\n############################################################\n\n\n")
       print("A RESPOSTA COMPLETA FOI: ")
       print(full_answer)
       print("\n\n++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n\n\n")
       chosen_letter = parse_answer((full_answer.replace(prompt, ""))) #Tira trechos do prompt da resposta
       if chosen_letter:
           for char in chosen_letter:
               if char.isalpha() and char.isupper():
                   chosen_letter = char
       else:
           chosen_letter = "X"
       end_time = time.time()
       elapsed_time = end_time - start_time
       start_time = time.time()

       question_prompts.append(prompt)

       token_size_questions.append(len(question['question'].split()))
       token_size_responses.append(len(full_answer.split()))

       temperatures.append(temperature)
       llm_full_answers.append(full_answer)
       llm_letters.append(chosen_letter)
       correct_answers.append(question['answers'])
       times.append(elapsed_time)

       is_correct = is_equal(question['answers'], chosen_letter)
       is_correct_list.append(is_correct)

       total_time += elapsed_time

       print(f"{question['Número']} / {len(questions)}",
      is_correct,
      "O tempo gasto foi:",
      elapsed_time, "chosen", chosen_letter, "correct", question["answers"], "\n")

    if dica:
        df = pd.DataFrame({'question_number': questions_df['Número'],
                           'question_body': questions_df['question'],
                           'full_model_name': model_name,
                           'model_family': model_name.split("/")[0],
                           'model_name': model_name.split("/")[1],
                           'llm_chosen_answer': llm_letters,
                           'correct_answer': correct_answers,
                           'is_correct': is_correct_list,
                           'legalTools':questions_df["legalTools"],
                           'Anulada?': questions_df['Anulada?'],
                             'question_prompt': question_prompts,
                             'response': llm_full_answers,
                             'token_size_question': token_size_questions,
                             'token_size_response': token_size_responses,
                             'elapsed_time_sec': times})
    else:
        df = pd.DataFrame({'question_number': questions_df['Número'],
                           'question_body': questions_df['question'],
                           'full_model_name': model_name,
                           'model_family': model_name.split("/")[0],
                           'model_name': model_name.split("/")[1],
                           'llm_chosen_answer': llm_letters,
                           'correct_answer': correct_answers,
                           'is_correct': is_correct_list,
                           'Anulada?': questions_df['Anulada?'],
                             'question_prompt': question_prompts,
                             'response': llm_full_answers,
                             'token_size_question': token_size_questions,
                             'token_size_response': token_size_responses,
                             'elapsed_time_sec': times})
    model_name = model_name.replace("/", ".")
    nome_arq = prova + f" {tipo_arquivo} - Modelo: " + model_name + " - "
    df.to_csv(f'{nome_arq}answers.csv')
    return total_time, is_correct_list.count(True), continuar


print("Functions defined!")

In [ ]:
def pequeno_teste_unload():
    # Teste para verificar a função unload_model
    
    model_name = "distilgpt2"  # Modelo pequeno para teste
    
    # Carrega o modelo e o tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")  # Carrega na GPU
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)
    
    # Gera um texto curto para testar se o modelo funciona
    output = pipe("Olá, mundo!", max_length=20)
    print(output)
    
    # Chama a função para descarregar o modelo e liberar a memória
    unload_model(model, pipe)

In [ ]:
def adicionandoColunas(df, dica):
    df_novo = df.copy()
    df_novo["Número"] = df_novo.index + 1
    df_novo["Anulada?"] = df_novo["answers"] == "*"
    if dica:
        df_novo = df_novo[["Número", "question", "comment", "legalTools", "answers", "Anulada?"]]
    else:
        df_novo = df_novo[["Número", "question", "comment", "answers", "Anulada?"]]
    return df_novo

In [ ]:
from collections import defaultdict

modelos = [
    # GEMINI
    "google/gemini-2.0-flash-exp",
    "google/gemini-1.5-pro",
    "google/gemini-1.5-flash",
    "google/gemini-1.5-flash-8b",
    "google/gemini-1.0-pro"
    
    # DEEPSEEK
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B", 
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B", 
    "deepseek-ai/DeepSeek-R1-Distill-Llama-8B", 
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B", 
    "deepseek-ai/deepseek-math-7b-instruct", 
    
   
    # QWEN
   "Qwen/Qwen2.5-1.5B", # small # 5 # feito 38, 39, 40, 41, 42
   "Qwen/Qwen2.5-7B", # 6 # feito 38, 39, 40, 41, 42
    # "Qwen/Qwen2-Math-7B-Instruct", # erro "greed decoding"
   "Qwen/Qwen2.5-3B-Instruct", # 7 # feito 38, 39, 40, 41, 42
    "Qwen/Qwen2.5-7B-Instruct", # 8 # feito 38, 39, 40, 41, 42
    "Qwen/Qwen2.5-Math-7B-Instruct", # 9 # feito 38, 39, 40, 41, 42
   "Qwen/Qwen2.5-Math-1.5B-Instruct", # 10 # feito 38, 39, 40, 41, 42
   "Qwen/Qwen2.5-Math-7B-PRM800K", # 11 # feito 38, 39, 40, 41, 42
   "Qwen/Qwen2.5-14B", # 12 # feito 38, 39, 40, 41, 42
   "Qwen/Qwen2.5-14B-Instruct", # 13 # feito 38, 39, 40, 41, 42
   "Qwen/Qwen2.5-Math-PRM-7B", # 14 # feito 38, 39, 40, 41, 42

    # Vicuna
    'lmsys/vicuna-7b-v1.5', # 15 # feito 38, 39, 40, 41, 42
   'lmsys/vicuna-13b-v1.5', # 16 feito 38, 39, 40, 41, 42
    
    # Bloom
    'bigscience/bloom-1b7', # 17 # feito 38, 39, 40, 41, 42
    # 'bigscience/bloom-7b1' # erro de caminho incorreto

    # LLEMMA
   "EleutherAI/llemma_7b", # 18 # feito 38, 39, 40, 41, 42

    # Falcon
   'tiiuae/falcon-7b-instruct', # 19 # feito 38, 39, 40, 41, 42
   'tiiuae/falcon-7b', # 20 # feito 38, 39, 40, 41, 42

    # Falcon2
    'tiiuae/falcon-11B', # 21 # feito 38, 39, 40, 41, 42

    # Falcon3
    'tiiuae/Falcon3-3B-Instruct', # 22 # feito 38, 39, 40, 41, 42
    'tiiuae/Falcon3-7B-Instruct', # 23 # feito 38, 39, 40, 41, 42
    'tiiuae/Falcon3-7B-Base', # 24 # feito 38, 39, 40, 41, 42
    'tiiuae/Falcon3-10B-Instruct', # 25 # feito 38, 39, 40, 41, 42
    'tiiuae/Falcon3-10B-Base', # 26 # Feito 38, 39, 40, 41, 42
    
     # PHI
   "microsoft/phi-1", # 27 # feito 38, 39, 40, 41, 42
   "microsoft/phi-1_5", # 28 # feito 38, 39, 40, 41, 42
   "microsoft/phi-2", # 29 # feito 38, 39, 40, 41, 42
   "microsoft/Phi-3-mini-4k-instruct", # 30 #feito 38, 39, 40, 41, 42
   "microsoft/Phi-3-medium-128k-instruct", # 31 #feito 38, 39, 40, 41, 42
   "microsoft/Phi-3.5-mini-instruct", # 32 # feito 38, 39, 40, 41, 42 
    # "microsoft/Phi-3.5-MoE-instruct", # NÃO FOI FEITOOOOOOOOOOO DEU SEM ESPAÇO
   "microsoft/phi-4", # 33 # feito 38, 39, 40, 41, 42

     # NuminaMath
     "AI-MO/NuminaMath-7B-CoT", # 34 # feito 38, 39, 40, 41, 42
     # "AI-MO/NuminaMath-7B-TIR-GPTQ", ERRO CONFIG BITSANDBYTES
     "AI-MO/NuminaMath-7B-TIR", # 35 # feito 38, 39, 40, 41, 42

     # Orca
     "microsoft/Orca-2-7b", # 36 # feito 38, 39, 40, 41, 42
     "microsoft/Orca-2-13b", # 37 # feito 38, 39, 40, 41, 42

     # Yi
    "01-ai/Yi-6B", # 38 # feito 38, 39, 40, 41, 42

     # Mistral
    # 'mistralai/Mathstral-7b-v0.1', # ACESSO RESTRITO
    # "mistralai/Mistral-7B-v0.1", # ACESSO RESTRITO
    # "mistralai/Mistral-7B-v0.3", # ACESSO RESTRITO
    # "mistralai/Mistral-7B-Instruct-v0.1", # ACESSO RESTRITO
    # "mistralai/Mistral-7B-Instruct-v0.2", # ACESSO RESTRITO
    # "mistralai/Mistral-Nemo-Instruct-2407", # ACESSO RESTRITO
    #  "mistralai/Mixtral-8x7B-Instruct-v0.1", # ACESSO RESTRITO
     "meta-math/MetaMath-Mistral-7B", # 39 # feito 38, 39, 40, 41, 42

     # Sabia
    "maritaca-ai/sabia-7b", # 40 # feito 38, 39, 40, 41, 42

    # LLmama
     # "meta-llama/Meta-Llama-3-70B", # ACESSO RESTRITO
   # "meta-llama/Meta-Llama-3-8B-Instruct", # ACESSO RESTRITO
    # "meta-llama/Llama-2-7b", # ACESSO RESTRITO
   # "meta-llama/Meta-Llama-3-8B", # ACESSO RESTRITO

    # Flan
   # "google/flan-t5-xxl", # CONFIGURAÇÃO DESCONHECIDA PARA AUTOMODELCAUSALLM 
   "google/flan-t5-small", # 41  # feito 38, 39, 40, 41, 42
    # "google/mt5-base", # CONFIGURAÇÃO DESCONHECIDA PARA AUTOMODELCAUSALLM
    
    # "google/gemma-2-2b", # ACESSO RESTRITO
    # "google/gemma-2-2b-it", # ACESSO RESTRITO
    #"google/gemma-2-9b", # ACESSO RESTRITO 
     # "google/gemma-2-9b-it", # ACESSO RESTRITO 
    #"google/gemma-2-27b", # ACESSO RESTRITO
    ]

print("LET'S GO!", len(modelos), " models.")
provas = {"oab38": [5], "oab39" : [], "oab40": [], "oab41": [], "oab42": []}
diretorio = "provas-oab-comentadas"
dica = False
caminho_original = "/kaggle/input/"
for prova, listaModelosUsados in provas.items():
    print(f"A prova é: {prova}")
    caminho_diretorio = os.path.join(caminho_original, diretorio)
    for arquivo in os.listdir(caminho_diretorio):
        if prova in arquivo:
            caminho_completo = os.path.join(caminho_diretorio, arquivo)
        else:
            print("ERROOOOOOOOOOO")
    print(f"O caminho completo é: {caminho_completo}")
    oab_exam_df = pd.read_csv(caminho_completo)
    oab_exam_df = adicionandoColunas(oab_exam_df, dica)
    for numero_modelo in listaModelosUsados:
        model = ""
        model_name = modelos[numero_modelo]
        print(f"Running {model_name}...")
        if model_name == 'google/gemini-1.5-flash' or  \
            model_name == 'google/gemini-1.5-flash-8b' or \
            model_name == "google/gemini-2.0-flash-exp" or \
            model_name == "google/gemini-1.5-pro" or \
            model_name == "google/gemini-1.0-pro":
            pipe = None
        else:
            pipe, model = load_model(model_name, True)
        dic = defaultdict()
        dic[0.7] = defaultdict(float)
        print(f"Answering questions using {model_name}, temperature {0.7} and quant={True}...")
        total_time, ctt_score, continuar = do_exam(oab_exam_df, model_name, True, pipe, 0.7, prova, dica)
        if continuar == False:
          print("Deu ruim acabou o funcionamento!!!!")
          break
        print(f"Finished {model_name} after {total_time/60} minutes. Score: {ctt_score}")
        if pipe != None:
            unload_model(model, pipe)
        print("###########", model_name)
print('FINISHED!')